# CLMR with Last.fm
このコードはLast.fmデータセットをCLMRを用いて楽曲の特徴量を抽出し，曲のメタデータとマージしてProtoMFに適応させるデータセットを作成するコードです．

# 環境構築

In [ ]:
# 必要なライブラリのインストール
!pip install librosa
!pip install torch torchvision
!pip install tqdm

# 必要なライブラリのインポート
import pandas as pd
import librosa
import os
import numpy as np
import torch
from torchvision import models
from tqdm import tqdm

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install spotipy python-dotenv pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 4.9 MB/s eta 0:00:00


# データセットのロード

In [ ]:
import pandas as pd

# CSVファイルの読み込み
listening_history_train = pd.read_csv('/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/listening_history_train.csv')
listening_history_test = pd.read_csv('/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/listening_history_test.csv')
listening_history_val = pd.read_csv('/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/listening_history_val.csv')
tracks = pd.read_csv('/content/drive/MyDrive/Master/research/data/lastfm/tracks.tsv', delimiter='\t')
users=pd.read_csv('/content/drive/MyDrive/Master/research/data/lastfm/users.tsv', delimiter='\t')
inter_dataset=pd.read_csv('/content/drive/MyDrive/Master/research/data/lastfm/inter_dataset.tsv', delimiter='\t')

In [ ]:
listening_history_test.head()

,old_user_id,old_item_id,timestamp,user_id,item_id
0,2,41515706,2020-03-20 12:09:50,1980,27086
1,36,17063812,2020-03-20 10:10:09,5960,1675
2,42,14940971,2020-03-17 13:36:22,5213,11687
3,44,21288474,2020-03-10 07:57:21,8315,24324
4,45,34835124,2020-03-13 12:00:04,7425,9983


In [ ]:
tracks.head()

,track_id,artist,track
0,138,Böhse Onkelz,!
1,155,Global Goon,!
2,159,Hellyeah,!
3,174,Kylie Minoise,!
4,193,Nogu Svelo!,!


In [ ]:
users.head()

,user_id,country,age,gender,creation_time
0,2,UK,35,m,2002-10-29 01:00:00
1,6,AT,28,n,2003-07-23 02:00:00
2,14,UK,48,m,2003-02-18 21:44:13
3,15,US,28,m,2003-02-24 03:30:33
4,20,NaN,-1,n,2003-03-19 13:18:50


In [ ]:
inter_dataset.head()

,user_id,track_id,album_id,timestamp
0,33738,10326542,13863140,2020-01-01 00:00:01
1,10000,38364127,13438862,2020-01-01 00:00:01
2,16026,40012596,18558010,2020-01-01 00:00:01
3,42410,14250568,8916172,2020-01-01 00:00:01
4,19519,36374119,4865313,2020-01-01 00:00:01


In [ ]:
# listening_history_test の 'item_id' と tracks の 'track_id' を基にマージ
merged_train_data = pd.merge(listening_history_train, tracks, left_on='old_item_id', right_on='track_id', how='left')
merged_val_data = pd.merge(listening_history_val, tracks, left_on='old_item_id', right_on='track_id', how='left')
merged_test_data = pd.merge(listening_history_test, tracks, left_on='old_item_id', right_on='track_id', how='left')

# マージ後のデータの確認
print(merged_val_data.head())

   old_user_id  old_item_id            timestamp  user_id  item_id  \
0            2     23449332  2020-03-17 07:14:44     1453    31594   
1           36     17012856  2020-03-16 16:50:49     7296    18528   
2           42     20468327  2020-03-17 10:28:34     4316    30060   
3           45      9408631  2020-03-13 10:55:44     6742    31111   
4           47      7373096  2020-03-19 22:09:18     7029     1729   

     track_id          artist                               track  
0  23449332.0     Code Orange                      Last Ones Left  
1  17012856.0           Lizzo  Good As Hell (feat. Ariana Grande)  
2  20468327.0       In Flames                       In Plain View  
3   9408631.0  Porridge Radio                            Circling  
4   7373096.0           Queen        Bohemian Rhapsody - 2011 Mix  


In [ ]:
# 結果を保存
merged_train_data.to_csv('/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/train_with_metadata.csv', index=False)
merged_val_data.to_csv('/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/val_with_metadata.csv', index=False)
merged_test_data.to_csv('/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/test_with_metadata.csv', index=False)

print('Merged data saved')

Merged data saved


In [ ]:
merged_train_data

,old_user_id,old_item_id,timestamp,user_id,item_id,track_id,artist,track
0,2,30937487,2020-02-20 07:10:29,1453,5054,30937487.0,TesseracT,Phoenix
1,2,25099053,2020-02-20 19:00:33,1453,14445,25099053.0,TesseracT,Luminary
2,2,13081182,2020-02-20 22:27:40,1453,17070,13081182.0,TesseracT,Dystopia
3,2,18298303,2020-02-20 22:37:26,1453,17148,18298303.0,TesseracT,Hexes
4,2,43063791,2020-02-21 07:01:39,1453,19194,43063791.0,TesseracT,Utopia
...,...,...,...,...,...,...,...,...
670294,120095,15181618,2020-03-03 12:20:47,7442,19495,15181618.0,The National,Fireproof
670295,120095,34195593,2020-03-03 12:24:02,7442,2560,34195593.0,Tony Anderson & Luke Atencio,Salt Lake City
670296,120095,14273952,2020-03-04 08:56:23,7442,11334,14273952.0,Neon Trees,Everybody Talks
670297,120095,44725239,2020-03-04 09:16:56,7442,25505,44725239.0,Generationals,"When They Fight, They Fight"


# APIを用いて音源ファイルを取得

音源データをSpotify APIを用いて取得しdownloaded_mp3sフォルダに保存

音源取得を行うデータセットをキーボード入力から受け取るので，コンソールの指示にしたがって操作を進めて下さい

In [ ]:
# 必要なライブラリをインポート
import os
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm
import re

# Spotify APIの認証
client_id = 'b72298f9c92343fe8e22be492c23a9c9'
client_secret = '184856f6edf54ec8badd48fe414b3296'
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# 音源ダウンロード関数
def download_mp3(url, output_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
        return False

# ファイル名に使えない文字を取り除く関数
def sanitize_filename(filename):
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

# Spotify API検索関数（レートリミット対応）
def search_spotify_with_retry(query, max_retries=5):
    retry_count = 0
    while retry_count < max_retries:
        try:
            results = sp.search(q=query, type='track', limit=1)
            return results
        except spotipy.exceptions.SpotifyException as e:
            if e.http_status == 429:  # レートリミットエラー
                retry_after = int(e.headers.get("Retry-After", 1))  # 秒数を取得
                print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
                time.sleep(retry_after)
            else:
                print(f"Spotify API error: {e}")
                return None
        except Exception as e:
            print(f"Unexpected error: {e}")
            return None
        retry_count += 1

    print("Max retries reached.")
    return None

# 音源ファイルをダウンロードする関数
def download_audio_files(csv_path, output_folder):
    import pandas as pd

    # CSVファイルを読み込む
    df = pd.read_csv(csv_path)

    # 出力フォルダを作成
    os.makedirs(output_folder, exist_ok=True)

    skip_count = 0  # スキップされた回数をカウント

    # tqdmを使って進捗バーを表示
    with tqdm(total=len(df), desc="Downloading MP3s") as pbar:
        for idx, row in df.iterrows():
            track_name = row['track']
            artist_name = row['artist']

            # 保存用のMP3ファイルパスを生成
            mp3_filename = sanitize_filename(f"{track_name} - {artist_name}.mp3")
            mp3_filepath = os.path.join(output_folder, mp3_filename)

            # 既にファイルが存在する場合はスキップ
            if os.path.exists(mp3_filepath):
                skip_count += 1  # スキップ回数をインクリメント
                pbar.set_postfix({'Skipped': skip_count})  # スキップ回数を進捗バーに表示
                pbar.update(1)
                continue

            # Spotifyで検索してプレビューURLを取得
            query = f'track:{track_name} artist:{artist_name}'
            results = search_spotify_with_retry(query)

            if results and results['tracks']['items']:
                track = results['tracks']['items'][0]
                preview_url = track.get('preview_url')

                if preview_url:
                    # MP3をダウンロード
                    if download_mp3(preview_url, mp3_filepath):
                        print(f"Downloaded: {mp3_filepath}")
                    else:
                        print(f"Failed to download: {mp3_filepath}")
                else:
                    print(f"No preview available for: {track_name} by {artist_name}")
            else:
                print(f"No results found for: {track_name} by {artist_name}")

            pbar.update(1)

# 実行するデータセットとフォルダを指定
csv_paths = {
    "train": '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/train_with_metadata.csv',
    "val": '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/val_with_metadata.csv',
    "test": '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/test_with_metadata.csv'
}
output_folder = '/content/drive/MyDrive/Master/research/data/lastfm/downloaded_mp3s'

# ユーザー入力で処理対象を選択
dataset_type = input("処理するデータセットを選択してください (val, train, test): ").strip().lower()

# 処理
if dataset_type in csv_paths:
    download_audio_files(csv_paths[dataset_type], output_folder)
else:
    print("無効な入力です。val, train, test のいずれかを入力してください。")

処理するデータセットを選択してください (val, train, test): train


CSVにmp3_pathとして音源ファイルを保存します。

APIに制限により音源が全てダウンロードできていない場合でも実行できます。

In [ ]:
import os
import pandas as pd
import re
from tqdm import tqdm  # tqdmをインポート

# ファイル名に使えない文字を取り除く関数
def sanitize_filename(filename):
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

# MP3ファイルの参照とCSV更新関数
def update_csv_with_mp3_paths(csv_path, mp3_folder, save_folder):
    # CSVを読み込み
    df = pd.read_csv(csv_path)

    # 'mp3_path' 列が存在しない場合は作成し、存在する場合は型をobjectに変更
    if 'mp3_path' not in df.columns:
        df['mp3_path'] = None
    else:
        df['mp3_path'] = df['mp3_path'].astype('object')  # 明示的にobject型に変更

    # MP3ファイルの参照
    mp3_files = {sanitize_filename(file): os.path.join(mp3_folder, file) for file in os.listdir(mp3_folder)}

    # tqdmで進捗バーを表示
    for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Updating mp3_path", unit="row"):
        track_name = row['track']
        artist_name = row['artist']

        # 保存用のMP3ファイル名を生成
        mp3_filename = f"{track_name} - {artist_name}.mp3"
        mp3_filename = sanitize_filename(mp3_filename)  # ファイル名を正規化

        # ファイルが存在する場合はパスを設定、存在しない場合はNULL
        if mp3_filename in mp3_files:
            df.at[idx, 'mp3_path'] = mp3_files[mp3_filename]
        else:
            df.at[idx, 'mp3_path'] = None

    # 結果を指定の保存先フォルダに新しいCSVとして保存
    os.makedirs(save_folder, exist_ok=True)  # 保存先フォルダが存在しない場合は作成
    updated_csv_path = os.path.join(save_folder, os.path.basename(csv_path).replace('.csv', '_with_mp3.csv'))
    df.to_csv(updated_csv_path, index=False)
    print(f"CSVファイルが更新されました: {updated_csv_path}")

# MP3フォルダとCSVファイルパスを指定
mp3_folder = '/content/drive/MyDrive/Master/research/data/lastfm/downloaded_mp3s'

train_csv_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/train_with_metadata.csv'
val_csv_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/val_with_metadata.csv'
test_csv_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/test_with_metadata.csv'

# 保存先のディレクトリ
save_folder = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/metadata_with_mp3/'

# 対象のCSVを更新
dataset_type = input("更新するデータセットを選択してください (val, train, test): ").strip().lower()

if dataset_type == 'val':
    update_csv_with_mp3_paths(val_csv_path, mp3_folder, save_folder)
elif dataset_type == 'train':
    update_csv_with_mp3_paths(train_csv_path, mp3_folder, save_folder)
elif dataset_type == 'test':
    update_csv_with_mp3_paths(test_csv_path, mp3_folder, save_folder)
else:
    print("無効な入力です。val, train, test のいずれかを入力してください。")

更新するデータセットを選択してください (val, train, test): train


Updating mp3_path: 100%|██████████| 670299/670299 [01:26<00:00, 7752.33row/s] 


CSVファイルが更新されました: /content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/metadata_with_mp3/train_with_metadata_with_mp3.csv


In [ ]:
# 以下、データセット確認用コードです。
train_df=pd.read_csv('/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/metadata_with_mp3/train_with_metadata_with_mp3.csv')
print("train_df")
print(train_df.head())

# mp3_pathがNaNの行の数を計算
nan_count = train_df['mp3_path'].isna().sum()

# データ全体の行数
total_count = train_df.shape[0]

# NaNの割合を計算
nan_ratio = nan_count / total_count * 100

# 結果を出力
print(f"mp3_pathがNaNの数: {nan_count}")
print(f"全体に占める割合: {nan_ratio:.2f}%")

# 特徴抽出

CLMRを用いて音源特徴を抽出します。

ファイルサイズが大きく、途中で実行を止めた場合でも再開時に途中から再開できるようにしています。

In [ ]:
mp3_dir = '/content/drive/MyDrive/Master/research/data/lastfm/downloaded_mp3s'

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
import torch
from torchvision import models

# 特徴量とtrack_idを格納するリスト
features = []
track_ids = []  # 追加: track_idを保存するリスト

# CLMRモデルの準備（必要に応じてモデルを選択）
model = models.resnet18(pretrained=True)  # ResNetの例
model.eval()  # 評価モードに設定

# データセットを選択するための入力を促す
dataset_type = input("抽出するデータセットを選択してください (train, val, test): ").strip().lower()

# 選択したデータセットに応じたCSVファイルのパスを設定
if dataset_type == 'train':
    metadata_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/metadata_with_mp3/train_with_metadata_with_mp3.csv'
elif dataset_type == 'val':
    metadata_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/metadata_with_mp3/val_with_metadata_with_mp3.csv'
elif dataset_type == 'test':
    metadata_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/metadata_with_mp3/test_with_metadata_with_mp3.csv'
else:
    print("無効な入力です。train, val, test のいずれかを入力してください。")
    exit(1)

# 出力ファイルのパス
output_path = f'/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/data_with_CLMR/features_{dataset_type}.csv'

# メタデータの読み込み
metadata = pd.read_csv(metadata_path)

# mp3_path列のデータ型を確認し、必要に応じて変換
metadata['mp3_path'] = metadata['mp3_path'].astype(str)

# 既存の特徴量ファイルをチェック
if os.path.exists(output_path):
    print(f"既存の特徴量ファイルが見つかりました: {output_path}")
    processed_features = pd.read_csv(output_path)
    processed_track_ids = set(processed_features['track_id'].astype(str))
    print(f"既に処理済みのtrack_id数: {len(processed_track_ids)}")
else:
    print(f"新しい特徴量ファイルを作成します: {output_path}")
    processed_features = pd.DataFrame()
    processed_track_ids = set()

# スキップカウントの初期化
skip_count = 0

# NaNの処理数カウントの初期化
nan_count = 0

# 一時保存の頻度（例: 100ファイルごと）
save_interval = 100
processed_since_last_save = 0

# 音楽ファイルのパスをループ処理
with tqdm(metadata.iterrows(), total=metadata.shape[0], desc="Processing files", unit="file") as pbar:
    for index, row in pbar:
        # track_idを文字列に変換
        track_id = str(row['track_id']).strip('"')  # track IDを文字列に変換

        # 既に処理済みのtrack_idをスキップ
        if track_id in processed_track_ids:
            skip_count += 1
            pbar.set_postfix({'skipped': skip_count, 'NaN count': nan_count})  # 進捗バーに両方を表示
            continue

        mp3_filename = row['mp3_path'].strip('"')  # mp3_pathを取得し、余分な引用符を取り除く

        # mp3_filenameがNaNでないことを確認
        if pd.isna(mp3_filename) or mp3_filename == 'nan':
            nan_count += 1  # mp3_filenameがNaNの場合、デフォルトの特徴量（ゼロベクトル）を設定
            features.append(np.zeros(13))  # 13次元のゼロベクトルをデフォルト特徴量として追加
            track_ids.append(track_id)  # track_idをリストに追加
            pbar.set_postfix({'skipped': skip_count, 'NaN count': nan_count})  # 進捗バーに両方を表示
            continue  # 無効なmp3_pathをスキップ

        file_name = os.path.join(mp3_dir, mp3_filename)

        try:
            # 音楽ファイルを読み込む
            y, sr = librosa.load(file_name, sr=None)

            # オーディオデータを適切な形式に変換
            y = np.expand_dims(y, axis=0)  # バッチ次元を追加

            # 特徴量を抽出
            with torch.no_grad():
                mfccs = librosa.feature.mfcc(y=y.flatten(), sr=sr, n_mfcc=13)
                features.append(mfccs.mean(axis=1))  # 平均値を計算して特徴量に追加
                track_ids.append(track_id)  # track_idをリストに追加

                # 進捗バー更新
                processed_since_last_save += 1

        except Exception as e:
            # エラーメッセージを出力
            print(f"Error processing {file_name}: {e}")

        # 一時保存
        if processed_since_last_save >= save_interval:
            # features は 2D リストなので、DataFrame を作成する際に列名を明示的に設定
            feature_columns = [f'feature_{i+1}' for i in range(13)]  # 列名を feature_1, feature_2, ..., feature_13 に設定
            new_features_df = pd.DataFrame(features, columns=feature_columns)
            new_features_df['track_id'] = track_ids

            # 既存ファイルとマージして保存
            if not processed_features.empty:
                combined_features_df = pd.concat([processed_features, new_features_df], ignore_index=True)
            else:
                combined_features_df = new_features_df

            combined_features_df.to_csv(output_path, index=False)
            print(f"一時保存しました: {output_path}")

            # 保存後に一時データをクリア
            features.clear()
            track_ids.clear()
            processed_features = combined_features_df
            processed_since_last_save = 0

# 最後に残りのデータを保存
if features:
    # features は 2D リストなので、DataFrame を作成する際に列名を明示的に設定
    feature_columns = [f'feature_{i+1}' for i in range(13)]  # 列名を feature_1, feature_2, ..., feature_13 に設定
    new_features_df = pd.DataFrame(features, columns=feature_columns)
    new_features_df['track_id'] = track_ids
    combined_features_df = pd.concat([processed_features, new_features_df], ignore_index=True)
    combined_features_df.to_csv(output_path, index=False)

print(f"特徴量抽出が完了しました。スキップしたファイル数: {skip_count}")
print(f"特徴量が{output_path}に保存されました。")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 145MB/s]


既存の特徴量ファイルが見つかりました: /content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/data_with_CLMR/features_train.csv
既に処理済みのtrack_id数: 31837


Processing files:  24%|██▍       | 163549/670299 [08:35<26:35, 317.56file/s, skipped=163550, NaN count=0]


KeyboardInterrupt: 

In [ ]:
# データセットを選択
dataset_type = input("マージしたいデータセットを選択してください (train, val, test): ").strip().lower()

# 選択したデータセットに応じたCSVファイルのパスを設定
if dataset_type == 'train':
    CLMR_data_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/data_with_CLMR/features_train.csv'
    metadata_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/train_with_metadata.csv'
elif dataset_type == 'val':
    CLMR_data_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/data_with_CLMR/features_val.csv'
    metadata_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/val_with_metadata.csv'
elif dataset_type == 'test':
    CLMR_data_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/data_with_CLMR/features_test.csv'
    metadata_path = '/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/dataset_with_metadata/test_with_metadata.csv'
else:
    print("無効な入力です。train, val, test のいずれかを入力してください。")
    exit(1)

features_df=pd.read_csv(CLMR_data_path)
# デバック用
# print("features_df")
# print(features_df.head())

metadata_df=pd.read_csv(metadata_path)
# デバック用
# print("metadata_df")
# print(metadata_df.head())

# メタデータとのマージ工程
# track_idを文字列型（object）に変換
metadata_df['track_id'] = metadata_df['track_id'].astype(str)
features_df['track_id'] = features_df['track_id'].astype(str)

# features_dfとval_dfをtrack_idでマージ
merged_df = pd.merge(metadata_df, features_df, on='track_id', how='left')

# 推薦に使用しないカラムの削除
columns_to_remove = ['track_id', 'artist', 'track']
merged_df = merged_df.drop(columns=columns_to_remove)

# 結果を確認
print(merged_df.head())

merged_df.to_csv(f'/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/data_with_CLMR/{dataset_type}_data_with_features.csv', index=False)
print(f'データセットが/content/drive/MyDrive/Master/research/ProtoMF/data/lfm2b-1mon/data_with_CLMR/{dataset_type}_data_with_features.csvに保存されました。')

# mp3_pathがNaNの行の数を計算
nan_count = merged_df['feature_1'].isna().sum()

# データ全体の行数
total_count = merged_df.shape[0]

# NaNの割合を計算
nan_ratio = nan_count / total_count * 100

# 結果を出力
print(f"feature_1がNaNの数: {nan_count}")
print(f"全体に占める割合: {nan_ratio:.2f}%")

# デバック用
# # merged_dfのサイズ
# print("merged_dfのサイズ")
# print(merged_df.shape)

features_dfに格納するデータセットを選択してください (train, val, test): train
   old_user_id  old_item_id            timestamp  user_id  item_id  \
0            2     30937487  2020-02-20 07:10:29     1453     5054   
1            2     25099053  2020-02-20 19:00:33     1453    14445   
2            2     13081182  2020-02-20 22:27:40     1453    17070   
3            2     13081182  2020-02-20 22:27:40     1453    17070   
4            2     18298303  2020-02-20 22:37:26     1453    17148   

    feature_1   feature_2  feature_3  feature_4  feature_5  feature_6  \
0 -132.121124  174.751068 -34.244244  30.576742 -14.549449  24.597698   
1  -49.962883  149.527802 -55.855682  48.107277 -34.124557  26.054760   
2  -86.538376  138.625122 -49.264763  48.222874 -21.839443  39.084221   
3  -86.538376  138.625122 -49.264763  48.222874 -21.839443  39.084221   
4  -72.292282  158.021698 -63.401085  41.785545 -25.203815  32.910114   

   feature_7  feature_8  feature_9  feature_10  feature_11  feature_12  \
0  -2.50